In [1]:
from datasets import Dataset, DatasetDict, Image
from sklearn.model_selection import train_test_split
from glob import glob
from huggingface_hub import notebook_login

In [2]:
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (manager-core).
Your token has been saved to C:\Users\USER\.cache\huggingface\token
Login successful


In [3]:
def create_dataset(image_paths, label_paths):
    dataset = Dataset.from_dict({"image": sorted(image_paths),
                                "annotation": sorted(label_paths)})
    dataset = dataset.cast_column("image", Image())
    dataset = dataset.cast_column("annotation", Image())

    return dataset

In [4]:
image = glob("classes_dataset/original_images/*")
label = glob("./classes_dataset/label_images_semantic/*")
print(len(image), len(label))

400 400


In [5]:
train_img, val_img = train_test_split(image, test_size=0.2, random_state=42)
train_label, val_label = train_test_split(label, test_size=0.2, random_state=42)
print("img :", len(train_img), len(val_img))
print("label :", len(train_label), len(val_label))

img : 320 80
label : 320 80


In [6]:
train_dataset = create_dataset(train_img, train_label)
validation_dataset = create_dataset(val_img, val_label)

In [7]:
dataset = DatasetDict({
    "train": train_dataset,
    "annotation": validation_dataset,
  }
)

In [8]:
dataset.push_to_hub("whyoke/segment_test")

Pushing split train to the Hub.


Map:   0%|          | 0/320 [00:00<?, ? examples/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split annotation to the Hub.


Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
import json
# simple example
id2label = {0: 'human', 1: 'tree', 2:"object"}
with open('id2label.json', 'w') as fp:
    json.dump(id2label, fp)